In [1]:
# -*- coding: utf-8 -*-
from hanspell import spell_checker
from UrlParserForKBO import UrlParserForKBO
from Parser_DaumKBO import Parser_DaumKBO
from Parser_KBO import Parser_KBO
from sklearn import tree
from sklearn.externals.six import StringIO
from TransformeTeamName import TransformeTeamName

import pandas as pd
import numpy as np
import MySQLdb
import random
import datetime
import os
import sys

In [2]:
# DecisionTree - 서두
path_Intro = './DecisionLearning//DecisionTree_sample.csv'
df_Intro = pd.read_csv(path_Intro,index_col=None,header=None,names=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum','context'])
score_Intro = np.array(df_Intro.reindex(columns=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum']))
context_Intro = np.array(df_Intro.get('context'))
contextSentence = [' ','승리하였다. ','접전끝에 승리를 하였다. ','역전에 성공했다. ','완승을 하였다. ','무승부로 끝이났다. ','영봉승을 하였다. ']
clf_Intro = tree.DecisionTreeClassifier(criterion='entropy')
clf_Intro = clf_Intro.fit(score_Intro,context_Intro)

In [3]:
def getClassifier():
    return clf_Intro.predict([x-y for x,y in zip(Parser_KBO.h_score[:13],Parser_KBO.a_score[:13])])[0]

In [4]:
def writeDate(parser_KBO):    
    return Parser_KBO.boxScore['date']

In [5]:
def writeHead(parser_KBO,parser_DaumKBO,contextClassifier):
#     문장뭉치
    sentenceGroup=[
        'at ascore vs ht hscore'
    ]
    
#     문장뽑기
    Head=sentenceGroup[random.randint(0,len(sentenceGroup)-1)]
    
#     문장만들기
    Head=sentence
    Head=Head.replace('at',Parser_KBO.a_team)
    Head=Head.replace('ht',Parser_KBO.h_team)
    Head=Head.replace('hscore',str(Parser_KBO.h_score[12]))
    Head=Head.replace('ascore',str(Parser_KBO.a_score[12]))
    
    return Head

In [ ]:
def writeIntro(Parser_KBO,Parser_DaumKBO,contextClassifier):
#     문장뭉치
    sentences1Gorup=(
        u'winTeam_name는 date stadium에서 열린 loseTeam_name와의 경기에서 winTeam_sum-loseTeam_sum 으로 context',
        u'date stadium에서 열린 away_name와 home_name와의 경기에서 away_sum-home_sum으로 winTeam_name이 context',
        u'away_name와 home_name의 경기가 date stadium에서 있었던 오늘 away_name는 away_sum, home_name는 home_sum로 winTeam_name가 경기에서 context',
        u'date stadium에서 있었던 away와 home과의 경기, winTeam이 winScore loseTeam이 loseScore로 winTeam_name이 context',
        u'홈팀 homeTeam_name과 어웨이팀 awayTeam_name 경기가 오늘 date 펼쳐졌다. stadium경기장에서 열린 이 경기에서 winTeam_name이 winTeam_sum : loseTeam_sum으로 loseTeam_name을 context',
    )
    sentences2Group=(
        u'이날 경기에서 winTeam_name winTeam_StarterPitcher선수가 투구수 피안타H 의 기록을 보여주며 호투를 하였다. ',
        u'이번 경기에서 winTeam_name의  winTeam_StarterPitcher선수가 winTeam_Start이닝을 호투하며, R실점 SO삼진을 잡아내며 승리를 이끌었다. ',
        u'오늘 경기간에 mvp_pitcher선수가 INN이닝을 던지고 SO삼진 ER자책 R실점 H피안타를 기록하여 팀의 승리에 기여하였다. '
    )
    MVP_PITCHER=None
    
#     문장뽑기
    Intro1=sentences1Gorup[random.randint(0,len(sentences1)-1)]
    Intro2=sentences2Group[random.randint(0,len(sentences1)-1)]
    
#     문장만들기

    #------- BEGIN Intro1-------#
    Intro1=Intro1.replace('winTeam_name',Parser_KBO['winTeam']['name'])
    Intro1=Intro1.replace('loseTeam_name',Parser_KBO['loseTeam']['name'])
    Intro1=Intro1.replace('winTeam_sum',str(Parser_KBO['winTeam']['sum']))
    Intro1=Intro1.replace('loseTeam_sum',str(Parser_KBO['loseTeam']['sum']))
    Intro1=Intro1.replace('home_sum',str(Parser_KBO['home']['sum']))
    Intro1=Intro1.replace('away_sum',str(Parser_KBO['away']['sum']))
    Intro1=Intro1.replace('stadium',Parser_DaumKBO['stadium'])
    Intro1=Intro1.replace('date',str(int(Parser_KBO['date'][4:6]))+u'월'+str(int(Parser_KBO.date[6:8]))+u'일'+Parser_KBO.date[8:])
    Intro1=Intro1.replace('home',Parser_KBO['home'])
    Intro1=Intro1.replace('away',Parser_KBO['away'])    
    
    # Intro1 예외처리
    #'context' -> 무승부
    if Parser_KBO['home']['sum']==Parser_KBO['away']['sum']:
        Intro1=Intro1.replace('context',context_sen_Intro[5].decode('utf-8'))    
    #'context' -> 영봉승
    elif Parser_KBO['home']['sum']==0 or Parser_KBO['away']['sum']==0:
        Intro1=Intro1.replace('context',context_sen_Intro[6].decode('utf-8'))    
    #'context'-> 나머지의 경우
    else:
        Intro1=Intro1.replace('context',context_sen_Intro[getClassifier(url)].decode('utf-8'))
    #------- END Intro1-------#

    #------- BEGIN Intro2-------#
    Intro2=Intro2.replace('winTeam_name',Parser_KBO['winTeam']['name'])
    Intro2=Intro2.replace('mvp_pitcher',Parser_KBO['winTeam']['pitRecord'].ix[row][u'선수명'])
    Intro2=Intro2.replace('ERA',Parser_KBO['winTeam']['pitRecord'].ix[row][u'평균자책점'])
    Intro2=Intro2.replace('INN',Parser_KBO['winTeam']['pitRecord'].ix[row][u'이닝'])
    Intro2=Intro2.replace('BN',Parser_KBO['winTeam']['pitRecord'].ix[row][u'타자'])
    Intro2=Intro2.replace('NP',Parser_KBO['winTeam']['pitRecord'].ix[row][u'투구수'])
    Intro2=Intro2.replace('SO',Parser_KBO['winTeam']['pitRecord'].ix[row][u'삼진'])            
    Intro2=Intro2.replace('ER',Parser_KBO['winTeam']['pitRecord'].ix[row][u'자책'])
    Intro2=Intro2.replace('R',Parser_KBO['winTeam']['pitRecord'].ix[row][u'실점'])
    Intro2=Intro2.replace('H',Parser_KBO['winTeam']['pitRecord'].ix[row][u'피안타'])
    #-------END Intro2-------#
    return Intro1+Intro2

In [8]:
# 결론작성
def writeConc(parser_KBO,parser_DaumKBO):
    # Conc
    Conclusion=Parser_KBO.a_status   
    
    return Conclusion

In [9]:
def getEmblem(parser_KBO):
    emblem=getInitialTeamName.get(Parser_KBO.winTeam)
    return emblem

In [10]:
# 본문작성
def writeMain(parser_KBO,parser_DaumKBO):
    url=url.replace('BoxScore','Situation')
    month=Parser_KBO.date[4:6]
    day=Parser_KBO.date[6:8]
    critical_Inning=getCriticalInning(date='2016'+month+day,awayTeam=unicode(Parser_KBO.a_team))
#     Main1 : about critical Inning
    Main1=u''
#     Main2 : about the flow of critical Inning
    Main2=u''
#     Main3 : about the keyPlayer(batter)
    Main3=u''
    
    
#     Main1 BEGIN
#     no critical Inning
    if len(critical_Inning)==0:
        sen=(
            u'',
            u'결정적인 이닝이 없던 무난한 경기였다. ',
            u'어느 이닝이 중요하다 뽑을수가 없는 경기였다'
            )
        sen=sen[random.randint(0,len(sen)-1)]
        
#         1 or more critical Exist
    else:
        inning=','.join([inn[0] for inn in critical_Inning])
        sen=(
            u'경기의 양상은 inning에서 판가름났다. ',
            u'승부는 inning이닝에서 갈리게 되었다. ',
            u'승부수는 바로 inning에서 갈라지게 되었다. ',
            u'양팀간의 승부는 inning에서 뚜렷하게 나타났다. ',
            u'두팀의 승부는 결국 inning에서 두드러지게 되었다. '
                )
        sen=sen[random.randint(0,len(sen)-1)]
        sen=sen.replace('inning',inning)
#         Main1 END
    Main1+=sen

#     Main2 BEGIN
    for inn in critical_Inning:
        Main2=u'inn 이닝에서 '
        Main2=Main2.replace(u'inn',inn[0])
#         open All Of Flow
        data=urllib2.urlopen(url+'#sms11')
        html=BeautifulSoup(data)
        cri=str.format('div.situation div#sms%02d > table'%(int(inn[0])))
#                              first = AWAY WIN
        if inn[1]=='first':
            cri=html.select(cri)[0]
#                              second= HOME WIN
        else:
            cri=html.select(cri)[1]
        
        hitContextList=[u'안타',u'루타',u'홈런']
        tmphitList=[]
        hitList=[]
        for line in cri.text.splitlines():
            line = line.strip()
            if line!='':
                tmphitList.append(line)
        for sen in tmphitList[2:]:
            hitList.append(sen.split(' ',2)[2])
        
        for sen in hitList:
#             print sen
            for context in hitContextList:
#                 print context
                if context in sen:
                    player=unicode(sen.split(' ',1)[0])
                    action=unicode(sen.split(' ',1)[1])
                    ex_sen=u'player선수의 action,'
                    ex_sen=ex_sen.replace(u'player',player) 
                    ex_sen=ex_sen.replace(u'action',action)
                    Main2+=ex_sen
                    break
                            
                    
        Main2+=u'등이 Parser_KBO.winTeam의 승리에 큰 기여를 하였다. '
        Main2=Main2.replace(u'Parser_KBO.winTeam',Parser_KBO.winTeam)
#         Main2 END

#         Main3 BEGIN
    Main3=u''
    year=Parser_KBO.date[:4]
    month=Parser_KBO.date[4:6]
    day=Parser_KBO.date[6:8]
    url='http://score.sports.media.daum.net/schedule/baseball/kbo/Main.daum?game_year=%s&game_month=%s'%(year,month)
    page=urllib2.urlopen(url)
    html=BeautifulSoup(page)
    
    s=unicode(html)
    date_index=s.find(str.format('<td class="time_date" rowspan="5">%d<span class="txt_day">')%(int(day)))
    s=s[date_index:]
    
    s=s[s.find(Parser_KBO.a_team):]
    serial_find_url='http://sports.media.daum.net/sports/gamecenter/'
    serial_start_index=s.find(serial_find_url)+len(serial_find_url)
    serial=s[serial_start_index:serial_start_index+8]
    
    url=str.format('http://m.sports.media.daum.net/m/sports/pack/3min/%s?lineup')%(serial)
    
    if os.name=='nt':
        driver=webdriver.PhantomJS(executable_path='./phantomjs.exe')
    else:
        driver=webdriver.PhantomJS(executable_path='./phantomjs')
    driver.get(url)
    html=driver.page_source
#         PhantomJS 드라이버 닫기
    driver.close()
#         Window에서는 PhantomJS프로세스가 남아있으므로 강제종료
    if os.name=='nt':
        os.system('taskkill /f /im phantomjs.exe')
    soup=BeautifulSoup(html)
    
    keyPlayerInfo=[]
    for team,css_select in zip((u'AWAY',u'HOME'),('td.position.away.key-player','td.position.home.key-player')):
        if soup.select_one(css_select)==None:
            continue
        sel=soup.select_one(css_select)
        info={}
        info[u'선수명']=sel.findNextSibling().text
        info[u'홈어웨이']=team
        
        if team==u'AWAY':
            keyPlayerBattingInfo_toDict=Parser_KBO.a_batRecord[Parser_KBO.a_batRecord[u'선수명']==info[u'선수명']]
            keyPlayerBattingInfo_toDict=keyPlayerBattingInfo_toDict.to_dict('records')[0]
        else:
            keyPlayerBattingInfo_toDict=Parser_KBO.h_batRecord[Parser_KBO.h_batRecord[u'선수명']==info[u'선수명']]
            keyPlayerBattingInfo_toDict=keyPlayerBattingInfo_toDict.to_dict('records')[0]
        
        info[u'득점']=keyPlayerBattingInfo_toDict[u'득점']
        info[u'타율']=keyPlayerBattingInfo_toDict[u'타율']
        info[u'타점']=keyPlayerBattingInfo_toDict[u'타점']
        info[u'타수']=keyPlayerBattingInfo_toDict[u'타수']
        info[u'안타']=keyPlayerBattingInfo_toDict[u'안타']
        keyPlayerInfo.append(info)
    
    for kPlayer in keyPlayerInfo:
    #     승리한팀에 키플레이어가 존재하는 경우
        if kPlayer[u'선수명'] in Parser_KBO.winTeam_batRecord[u'선수명'].values:
            Main3=(
                u'Parser_KBO.winTeam의 kPlayer 선수가 Movement 를 기록하여 팀의 키플레이어 역할을 해주었다. ',
                u'Parser_KBO.winTeam kPlayer 선수가 Movement 를 기록 핵심타자 역할을 해주었다. ',
                u'kPlayer선수가 Movement를 하며, Parser_KBO.winTeam의 핵심적인 득점원이 되어주었다. '
            )
            Main3=Main3[random.randint(0,len(Main3)-1)]
            Main3=Main3.replace(u'Parser_KBO.winTeam',Parser_KBO.winTeam)
            Main3=Main3.replace(u'kPlayer',kPlayer[u'선수명'])
            Movement=u'Deukjum Tayul Tajum Tasu Anta'
            Movement=Movement.replace(u'Deukjum',u'득점 '+kPlayer[u'득점'])
            Movement=Movement.replace(u'Tayul',u'타율 '+kPlayer[u'타율'])
            Movement=Movement.replace(u'Tajum',u'타점 '+kPlayer[u'타점'])
            Movement=Movement.replace(u'Tasu',u'타수 '+kPlayer[u'타수'])
            Movement=Movement.replace(u'Anta',u'안타 '+kPlayer[u'안타'])
            Main3=Main3.replace(u'Movement',Movement)

    #     패배한팀에 키플레이어가 존재하는 경우
        else:
            Main3=(
                u'',
                u'Parser_KBO.loseTeam의 kPlayer선수가 Movement 를 하였지만 아쉽게도 팀을 승리로 이끌기에는 부족하였다. ',
                u'kPlayer선수 또한 Movement를 하였지만 Parser_KBO.loseTeam의 승리를 돕기에는 부족했다. ',
                u'Parser_KBO.loseTeam의 kPlayer선수가 Movement 를 하며 열심히 방망이질을 했지만 아쉬운 결과였다. '
                )
            Main3=Main3[random.randint(0,len(Main3)-1)]
            Main3=Main3.replace(u'Parser_KBO.loseTeam',Parser_KBO.loseTeam)
            Main3=Main3.replace(u'kPlayer',kPlayer[u'선수명'])
            Movement=u'Deukjum Tayul Tajum Tasu Anta'
            Movement=Movement.replace(u'Deukjum',u'득점 '+kPlayer[u'득점'])
            Movement=Movement.replace(u'Tayul',u'타율 '+kPlayer[u'타율'])
            Movement=Movement.replace(u'Tajum',u'타점 '+kPlayer[u'타점'])
            Movement=Movement.replace(u'Tasu',u'타수 '+kPlayer[u'타수'])
            Movement=Movement.replace(u'Anta',u'안타 '+kPlayer[u'안타'])
            Main3=Main3.replace(u'Movement',Movement)
     
    return Main1+Main2+Main3

In [5]:
startDate='20160824'
endDate='20160824'
urlParserForKBO = UrlParserForKBO(startDate,endDate)

In [12]:
# MySQL conf
con=MySQLdb.connect(host='218.150.181.131',user='root',passwd='1234',db='link10th',charset='utf8', use_unicode=True)

for url in urlParserForKBO.urlList:
    parser_KBO=Parser_KBO(url)
    parser_DaumKBO=Parser_DaumKBO(parser_KBO.boxScore['date'],parser_KBO.boxScore['awayTeam'])
    try:
        date=writeDate(parser_KBO)
        contextClassifier=getClassifier()
        Head=writeHead(parser_KBO,parser_DaumKBO,contextClassifier)
        Intro=writeIntro(parser_KBO,parser_DaumKBO,contextClassifier)
        Main=writeMain(parser_KBO,parser_DaumKBO)
        Conc=writeConc(parser_KBO,parser_DaumKBO)
        emblem=getEmblem(parser_KBO)
    except:
        sys.stderr.write(unicode.format(u'********** 날짜 : %s %s와 %s의 경기 파싱오류 ********** '%(Parser_KBO.date,Parser_KBO.a_team,Parser_KBO.h_team)))
    
    #교정부분
#     Head = spell_checker.check(Head)
#     Intro = spell_checker.check(Intro)
#     Main = spell_checker.check(Main)
#     Conc = spell_checker.check(Conc)
#     Head = Head.checked
#     Intro = Intro.checked
#     Main = Main.checked
#     Conc = Conc.checked
     
    cursor=con.cursor()
    cursor.execute(
    '''
    INSERT INTO Article(date,Head,Intro,Main,url,emblem,winCount,loseCount,drawCount)
    VALUES (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\'),\'%s\'),\'%s\'))
    '''
    %(date,Head,Intro,Main,url,emblem,winCount,loseCount,drawCount)

    )
    # MySQL disconnect
    con.commit()
    cursor.close()
con.close()

C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\GoHyunyoung98\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\utils\valida